In [1]:
#!pip install biopython --upgrade
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from Bio.PDB import PDBParser
from Bio.PDB.SASA import ShrakeRupley
parser = PDBParser()
parser = PDBParser(PERMISSIVE=1)


filename_Config1 =r'C:\Users\bashc\Downloads\2pzs_clean.pdb'

maxRadius = 11 #nm max distance considered between two residues
 
showPairs = True  #draw in the pair lines between the redox pairs (chimeraX draws extra lines in, so expect a few pairs)

outputReorgScriptFile = r'D:\PythonProj\MDSimulations\reorg.cxc'
outputTimesScriptFile = r'D:\PythonProj\MDSimulations\transferTimes.cxc'

redoxEnergies = {'TRP': .85, #eV
                 'TYR': 1.05, #1.08,
                 #'PHE': 1.15,  #  This is just a guess value from a few papers (Charge Transfer in Model Peptides: Obtaining Marcus Parameters from Molecular Simulation)
                 'CYS': .3, # numbers on this seem to range from .23 up to 1.12. picking 
                 } 

#uses https://www.biorxiv.org/content/10.1101/2023.06.12.544705v1.full.pdf to calculate the reorganization energy.  Multi heme parameters are used
#results will vary 

#this will output a few cxc files that can be used to color chimeraX models
#to use, open the pdb file in chimeraX, then open the script in chimeraX


In [2]:
def ColorResidues(redox,  parameter):    
    minLambdaExpect = 100000
    maxLambdaExpect = -100000
    for i in range(0, len(redox)):
        if (redox[i][parameter] < minLambdaExpect):
            minLambdaExpect = redox[i][parameter]
        if (redox[i][parameter] > maxLambdaExpect):
            maxLambdaExpect = redox[i][parameter]


    for i in range(0, len(redox)):
        lambdaExpect = redox[i][parameter]
        lambdaExpect = (lambdaExpect - minLambdaExpect) / \
            (maxLambdaExpect - minLambdaExpect)

        if lambdaExpect < .2:
            redox[i]['color'] = 'blue'
        elif lambdaExpect < .4:
            redox[i]['color'] = 'green'
        elif lambdaExpect < .8:
            redox[i]['color'] = 'yellow'
        elif lambdaExpect < 1:
            redox[i]['color'] = 'orange'
        else:
            redox[i]['color'] = 'red'
        
    good="select "       

    for x in redox:
        good += f"/{x['chain']}:{x['residue']} "
    good+= "\nshow sel atoms\n"

    colors = list(set( [ x['color'] for x in redox]    ))
    for color in colors:
        good += "select "
        for x in [c for c in redox if c['color']==color]:
            good += f"/{x['chain']}:{x['residue']} "
        good += f"\ncolor sel {color}\n "       
    return good     

def GetSideAtoms(residue ):
    atoms=[]
    cc=0
    for atom in  residue.get_atoms():
        coords =np.array( atom.get_coord())/10.0 #convert to nm
        element = atom.get_id()[0]
        if element in  '123456' :
            element = atom.get_id()[1]
        #oxygen and thiols are used to determine jump distance
        name = atom.get_name()
        if not ((name in ['CA','CB','O','C']) or (cc==0 and name=='N')) :
            atoms.append([element,coords,atom.get_name()])
         
    return atoms

def EstimateReorg(residue1,residue2, minDist):
    alpha = 5.18  #cheating to mostly use the values from Guberman-Pfeffer's paper, no fitting was used to adapt these to amino acids other than dividing by 4
    beta = 0.016
    Rd = 2
    Ra = 2
    Eopt = 1.84
    
    totalSasa = residue1['sasa'] +residue2['sasa']
    Es = alpha + (beta * totalSasa)
    M = (1/Eopt) - (1/Es)
    R = (1/((2*Rd)/0.53)) + (1/((2*Ra)/0.53)) - (1/(minDist/0.53))
    return ( ((-1)**2) * (M) * (R) * (27.2114))/4

def LoadPDB(filename):
    structure = parser.get_structure('x', filename_Config1)
    sr = ShrakeRupley()
    sr.compute(structure, level="R")

    redoxActive=[x[0] for x in redoxEnergies.items()]

    residues = []
    # walk through chains and get all residues, COM, Solvent Accessible Surface Area (SASA) and side atoms
    for model in structure:
        for chain in model:
            for residue in chain.get_residues():
                coordsA = np.array(residue.center_of_mass())
                atoms = GetSideAtoms(residue)
                residues.append({'model': model.id,
                                'chain': chain.id,
                                'residue': residue.get_id()[1],
                                'amino': residue.get_resname(),
                                'sasa': residue.sasa,
                                'atoms': atoms,
                                '<lambda>': -3,
                                '<time_forward>': -10, 
                                'color' : 'dim gray',
                                'lambdas': [],
                                'lambda_pairs': [],
                                'energy_rate_forward': [],
                                'com': coordsA})
    return redoxActive, residues

def TimeRate(residue1,residue2,minDist):
    reorgE_EV = EstimateReorg(residue1,residue2,minDist) 
         

    hbar_eV = 6.582119569e-16  # eV*s
    kbT_eV = 8.617333262145E-5*300  # eV
   
    beta2 = 1/(kbT_eV)
    attemptFrequency = 2*np.pi/hbar_eV / \
        np.sqrt(4*np.pi * (reorgE_EV)*kbT_eV)   # attempt prefactor 2 pi/ hbar
    
    gammaV= -1.4
    distance_rate = attemptFrequency * np.exp(gammaV * minDist) 
                    
    dE = redoxEnergies[residue1['amino']] - redoxEnergies[residue2['amino']]
        
                # get the forward rate
    dF_F = (reorgE_EV + dE)**2/(4*reorgE_EV)
    
    preVibrate = 2.7/7* np.exp(gammaV/2 * minDist)
    preG= (.01e-9)* np.pi*preVibrate**2/hbar_eV 
    g_F=preG/ np.sqrt(np.abs(reorgE_EV* dF_F))
    
   
    energy_rate_forward = np.exp(-beta2 * dF_F)
    return distance_rate * energy_rate_forward /(1+g_F)   ,reorgE_EV

In [3]:
redoxActive, residues = LoadPDB(filename_Config1)

redox = [x for x in residues if x['amino'] in redoxActive]

pairDistance ={}
for i in range(0, len(redox)):
    for j in range(i+1, len(redox)):
        minDist = 1000
        atomPair = []
        for atoms in redox[i]['atoms']:
            for atoms2 in redox[j]['atoms']:
                d = np.linalg.norm(atoms[1] - atoms2[1])*10
                if d < minDist:
                    minDist = d
                    atomPair = {'select0': f"/{redox[i]['chain']}:{redox[i]['residue']}@{atoms[-1]}",
                                'select1': f"/{redox[j]['chain']}:{redox[j]['residue']}@{atoms2[-1]}"}
                    if i not in pairDistance:
                        pairDistance[i] = {}
                        
                    pairDistance[i][j]= {'d':d,'pair':atomPair}

        if minDist > maxRadius:
            continue
        
        Lambda = EstimateReorg(redox[i], redox[j], minDist)
        
      
        
        redox[i]['lambdas'].append(Lambda)
        redox[i]['lambda_pairs'].append([j, Lambda, atomPair])
        redox[j]['lambdas'].append(Lambda)
        redox[j]['lambda_pairs'].append([i, Lambda, atomPair])

r1=""
r2=""
for i in range(0, len(redox)):
    lambdas =[x for x in  redox[i]['lambdas'] if x!=0]
    if (redox[i]['residue'] == 128 or redox[i]['residue'] == 137):
        print(redox[i]['residue'],lambdas)
    redox[i]['<lambda>'] = np.mean( lambdas)
    redox[i]['min_lambda'] = np.min(lambdas)
    if showPairs:
        bestPair = np.argmin( [x[1] for x in redox[i]['lambda_pairs']] )
        bestPair = redox[i]['lambda_pairs'][bestPair]
        #contacts /a:8@CA restrict /a:180@CA distanceOnly 15 reveal true name aros0
        r1 += " " + bestPair[-1]['select0']
        r2 +=" " + bestPair[-1]['select1']
        
if showPairs:        
    connects = f"contacts {r1} restrict {r2} distanceOnly {maxRadius} reveal true name aros\n"
else :
    connects = ""

goodMean = ColorResidues(redox,  '<lambda>')    

#get path 
path = os.path.dirname(outputReorgScriptFile)
#get filename 
sFile =  os.path.basename(outputReorgScriptFile)

with open(os.path.join(path, "mean_" + sFile), "w") as file:
    file.write(goodMean + '\n' + connects)
    
goodMin = ColorResidues(redox,  'min_lambda')     
with open(os.path.join(path, "min_" + sFile), "w") as file:
     file.write(goodMin + '\n' + connects)

In [4]:

distanceGraph = []
for i in range(0, len(residues)):
    residues[i]['energy_rate_forward'] = []
    residues[i]['<time_forward>'] = -10
    

for i in range(0, len(redox)):
    for j in range(i+1, len(redox)):
        minDist = pairDistance[i][j]['d']
        atomPair =  pairDistance[i][j]['pair']
        
        if minDist > maxRadius:
            continue
        
        k_NA_forward,L=TimeRate(redox[i], redox[j],minDist)
        
        distanceGraph.append([minDist, k_NA_forward])
        if k_NA_forward >0:
            redox[i]['energy_rate_forward'].append(k_NA_forward)
            redox[j]['energy_rate_forward'].append(k_NA_forward)


for i in range(0, len(redox)):
    min = np.sum(redox[i]['energy_rate_forward'])   
    redox[i]['<time_forward>'] =np.log( 1/  min)
    
    
good = ColorResidues(redox,  '<time_forward>')      

with open(outputTimesScriptFile, "w") as file:
    file.write(good+ '\n' + connects)